In [1]:
import pandas as pd 
from Response_Spectrum import *
from sympy import symbols, Eq, Function,UnevaluatedExpr, Mul
from sympy import *

#init_printing()
from sympy import Piecewise, nan
import numpy as np

0.75


In [2]:
def round_expr(expr, num_digits=2):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(Number)})

def round_equation(eq, num_digits=2):
    lhs = eq.lhs
    rhs = eq.rhs
    rounded_rhs = round_expr(rhs, num_digits)
    return Eq(lhs, rounded_rhs)

In [3]:
x=symbols('x')
expr=1/sin(x) 
f=lambdify(x, expr) 
display(expr)
f(2)


1/sin(x)

1.0997501702946164

In [4]:
RS_params=AASHTO(T=1, PGA=0.4,S_S=0.75,S_1=0.2,SiteClass="B")
# Convert the first element of the array to a numpy number
#RS_params[0].item()
RS_params


(array([0.2]), array(1.), array(1.), array(1.), 0.4, 0.75, 0.2)

In [13]:
def Iteration(m,n,n_c,W_SS, W_PP,W,K_sub,angle_skew,S_D1, epsilon,d=2.0):
    
    """
    m: Number of supports
    
    n: Number of girders per support
    
    n_c=[0,3,3,0]: Number of columns per support
    ** abutment1, 2 there are no columns
        number of columns = 3 in each pie 1 and 2 
        
    
    W_SS: Weight of superstructure including railings, curbs,and barriers to the permanent loads
    
    W_PP: Weight of piers participating with superstructure in dynamic response
    
    W_eff= W_SS + W_PP: Effective weight
    
    W=[W_1,W_2,...,W_m]: Weight of superstructure at each support
    
    K_sub=[K_sub_abut1, K_sub_pie1, K_sub_pie2, K_sub_abut2]: Stiffness of each support in both longitudinal and transverse directions of the bridge
    ** For the abutments, take Ksub,j to be a large number, say 10,000 kips/in.
    
    angle_skew: Angle of skew
    
    PGA,S_1, S_S: Acceleration coefficients for bridge site are given in design statement
    
    SiteClass:  "A", "B", "C","D","E"
    
    epsilon: tolerance
    
    d: set initial guess for the first iteration
 
    """
    

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # B2.1.1—Step A: Identifty Bridge Data
    ## B2.1.1.2—Step A2: Seismic Hazard
    
    
    # B2.1.2—STEP B: ANALYZE BRIDGE FOR EARTHQUAKE LOADING IN LONGITUDINAL DIRECTION
    ## B2.1.2.1—STEP B1: SIMPLIFIED METHOD
    ### B2.1.2.1.1—Step B1.1: Initial System Displacement and Properties
        
    #%% Assume that the initial value of displacement d approximates 2.0
    #d=2.0
    i=1
    
    data={}
  
    while True:
        
        #print(f'iteration:{i} ')
        #print(f'd={d}')
        
        #%% Calculate characteristic strength, Q_d
        Q_d=0.05*W_SS

        #%% Calculate Post-yield stiffness, K_d
        K_d=0.05*(W_SS/d)

        ### B2.1.2.1.2—Step B1.2: Initial Isolator Properties at Supports

        #%% Calculate the characteristic strength, Q_dj

        Q_dj=[Q_d*(W[j]/W_SS) for j in range(m)]

        #%% Calculate postelastic stiffness, K_dj

        K_dj= [K_d*(W[j]/W_SS) for j in range(m)]
      

        ### B2.1.2.1.3—Step B1.3: Effective Stiffness of Combined Pier and Isolator System

        #%% Calculate alpha_j

        alpha_j = [(K_dj[j]*d+Q_dj[j])/(K_sub[j]*d-Q_dj[j]) for j in range(m)]

        #%% Calculate the effective stiffness, K_effj

        K_effj=[(alpha_j[j]*K_sub[j])/(1+alpha_j[j]) for j in range(m)]

        ### B2.1.2.1.4—Step B1.4: Total Effective Stiffness

        #%% Calculate the total effective stiffness, Keff, of the bridge:

        K_eff= sum(K_effj)
        #print(f'K_eff: {K_eff}')

        ### B2.1.2.1.5—Step B1.5: Isolation System Displacement at Each Suppor

        #%% Calculate the displacement of the isolation system, d_isolj

        d_isolj=  [d/(1+ alpha_j[j]) for j in range(m)]
        
        #print(f'd_isolj: {d_isolj}')

        ### B2.1.2.1.6—Step B1.6: Isolation System Stiffness at Each Support

        #%% Calculate the effective stiffness of the isolation system at support “j”, Kisol,j, for all supports

        K_isolj=[Q_dj[j]/d_isolj[j]+ K_dj[j] for j in range(m)]

        ### 2.1.2.1.7—Step B1.7: Substructure Displacement at Each Support, d_subj

        d_subj= [d- d_isolj[j] for j in range(m)]

        ### B2.1.2.1.8—Step B1.8: Lateral Load in Each Substructure Support

        #%% Calculate the shear at support “j”, Fsub,j, for all supports:

        F_subj= [K_sub[j]* d_subj[j] for j in range(m)]

        ### B2.1.2.1.9—Step B1.9: Column Shear Force at Each Support

        F_coljk=[]

        for j in range(m):
            if n_c[j]==0:
                val=0
            else:
                val= [F_subj[j]/n_c[j]]

            F_coljk.append(val)

        ### B2.1.2.1.10—Step B1.10: Effective Period and Damping Ratio

        #%% Calculate the effective period, T_eff 

        W_eff= W_SS + W_PP #  Effective weight, W_eff
        g=386.4 # (in./s^2) or 9.815(m/s^2)

        T_eff=2*np.pi* (W_eff/(g*K_eff))**(1/2)
        #print(f'T_eff: {T_eff}')

        #%% Calculate the viscous damping ratio, ξ , of the bridge

        d_yj=[0]*m # taking d_yj=0

        numerator=2*sum([Q_dj[j]*(d_isolj[j]-d_yj[j]) for j in range(m)])

        denominator=np.pi*sum([K_effj[j]*(d_isolj[j]+d_subj[j])**2 for j in range(m)])

        xi= numerator/denominator

        ### B2.1.2.1.11—Step B1.11: Damping Factor

        #%% Calculate the damping factor, B_L

        B_L=Piecewise(((xi/0.05)**0.03,xi<0.3),(1.7, xi>=0.3))

        #%%  Calculate the displacement, d_new

        d_new=(9.79*S_D1*T_eff)/B_L
        
        #print(f'd_new: {d_new}')
        
        df = pd.DataFrame({"Pier": ["Abut1", "Pier1", "Pier2", "Abut2"],
                           "$$W_j$$": W,
                           "$$Q_{d,j}$$":Q_dj,
                           "$$K_{sub,j}$$": K_sub,
                           "$$K_{d,j}$$":K_dj,
                           "$\\alpha_j$": alpha_j, 
                           "$$K_{eff,j}$$": K_effj,
                           "$$d_{isol,j}$$":d_isolj,
                           "$$K_{isol,j}$$": K_isolj,
                           "$$d_{sub,j}$$": d_subj,    
                           "$$F_{sub,j}$$":F_subj ,
                           "$$T_{eff}$$": T_eff,
                           "$$K_{eff}$$":K_eff,
                           "$$\\xi$$":xi,
                           "$$B_{L}$$": B_L})
        
        data[i]=df

        #%%%%%%%%%%%%%%%%%%%%%
        #%% Calculate the diference, abs(d_new-d) 

        difference=abs(d_new-d)

        #%% Check the condition:

        if difference> epsilon:
            d=d_new
            i+=1

        else:
            break
    print(f'Numbers of iteratations: {i}')
    
    return data
     

In [14]:
# Example 1.0:
m=4
n=6
n_c=[0,3,3,0]
W_SS=650.52
W_PP=107.16
W=[44.95,280.31,280.31,44.95]
K_sub=[10000,172.0, 172.0,10000]
angle_skew=0
#%%%%%%
PGA=0.40
S_1=0.20
S_S=0.75
SiteClass="B"


In [15]:
### Calculate Response Spectrum Parameters: C_sm, F_pga, F_a, F_v, A_S, S_DS,S_D1
C_sm, F_pga, F_a, F_v, A_S, S_DS,S_D1=AASHTO(T=1, PGA=PGA,S_S=S_S,S_1=S_1,SiteClass=SiteClass) 

print(f'C_sm={C_sm}')
print(f'F_pga={F_pga}')
print(f'F_a={F_a}')
print(f'F_v={F_v}')
print(f'A_S={A_S}')
print(f'S_DS={S_DS}')
print(f'S_D1={S_D1}')

C_sm=[0.2]
F_pga=1.0
F_a=1.0
F_v=1.0
A_S=0.4
S_DS=0.75
S_D1=0.2


In [24]:
epsilon=0.01
d=2
data=Iteration(m,n,n_c,W_SS, W_PP,W,K_sub,angle_skew,S_D1, epsilon, d)


Numbers of iteratations: 5


In [25]:
# Round the values in each column to n decimal places
def round_values(x,n):
    try:
        return round(x, n)
    except:
        return x
    
# Apply the e-4 formatting to numeric columns
def scientific_format(x):
    try:
        return f'{float(x):.2e}'
    except:
        return x
    
concatenated_df = pd.DataFrame()

# Concatenate the DataFrames from each iteration
for i, df in data.items():
    df['Iteration'] = i  # Add a new column for iteration number
    concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)

for col in concatenated_df.columns:
    if col=="$\\alpha_j$":
        concatenated_df[col] = concatenated_df[col].apply(scientific_format)
    else:
        concatenated_df[col] = concatenated_df[col].apply(round_values,n=2)
#concatenated_df.set_index(['Iteration',"$$T_{eff}$$","$$K_{eff}$$","$$\\xi$$", "$$B_{L}$$", 'Pier'], inplace=True)



In [26]:
concatenated_df

,Pier,$$W_j$$,"$$Q_{d,j}$$","$$K_{sub,j}$$","$$K_{d,j}$$",$\alpha_j$,"$$K_{eff,j}$$","$$d_{isol,j}$$","$$K_{isol,j}$$","$$d_{sub,j}$$","$$F_{sub,j}$$",$$T_{eff}$$,$$K_{eff}$$,$$\xi$$,$$B_{L}$$,Iteration
0,Abut1,44.95,2.25,10000.0,1.12,2.25e-04,2.25,2.0,2.25,0.0,4.49,1.57,31.43,0.31,1.70,1
1,Pier1,280.31,14.02,172.0,7.01,8.49e-02,13.47,1.84,14.61,0.16,26.93,1.57,31.43,0.31,1.70,1
2,Pier2,280.31,14.02,172.0,7.01,8.49e-02,13.47,1.84,14.61,0.16,26.93,1.57,31.43,0.31,1.70,1
3,Abut2,44.95,2.25,10000.0,1.12,2.25e-04,2.25,2.0,2.25,0.0,4.49,1.57,31.43,0.31,1.70,1
4,Abut1,44.95,2.25,10000.0,1.24,2.49e-04,2.49,1.81,2.49,0.0,4.49,1.49,34.65,0.31,1.70,2
5,Pier1,280.31,14.02,172.0,7.75,9.44e-02,14.84,1.65,16.24,0.16,26.82,1.49,34.65,0.31,1.70,2
6,Pier2,280.31,14.02,172.0,7.75,9.44e-02,14.84,1.65,16.24,0.16,26.82,1.49,34.65,0.31,1.70,2
7,Abut2,44.95,2.25,10000.0,1.24,2.49e-04,2.49,1.81,2.49,0.0,4.49,1.49,34.65,0.31,1.70,2
8,Abut1,44.95,2.25,10000.0,1.31,2.61e-04,2.61,1.72,2.61,0.0,4.49,1.46,36.31,0.31,1.70,3
9,Pier1,280.31,14.02,172.0,8.14,9.94e-02,15.55,1.57,17.09,0.16,26.76,1.46,36.31,0.31,1.70,3


In [27]:
import os 
current_path=os.getcwd()
folder= "Data_Examples"
folder_path=os.path.join(current_path,folder)
file_name= "Example_1.0.xlsx"
file_path = os.path.join(folder_path, file_name)
file_path

'/Users/namnguyen/Desktop/GitHub_CFC/BASE-ISOLATION/Data_Examples/Example_1.0.xlsx'

In [23]:

concatenated_df.to_excel(file_path, index=False)